<span style="font-size:2.5em">3. 단어 임베딩(Word Embedding)-1</span>

이 자료는 다음 자료들을 기반으로 수정된 자료입니다.<br>
- [위키독스 딥 러닝을 이용한 자연어 처리 입문, 04. 카운트 기반의 단어 표현(Count based word Representation)](https://wikidocs.net/22650)

# 3.1 Bag of Words

In [1]:
import sys
!{sys.executable} -m pip install konlpy

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kikim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### BoW 예 

In [3]:
from konlpy.tag import Okt

okt = Okt()

In [23]:
def build_bag_of_words(document):
    # 마침표(.) 제거
    document = document.replace('.', '')
    tokenized_document = okt.morphs(document)

    word_to_index = {}
    bow = []

    for word in tokenized_document:  
        if word not in word_to_index.keys():
            word_to_index[word] = len(word_to_index)  
            # BoW에 전부 기본값 1을 넣는다.
            bow.insert(len(word_to_index) - 1, 1)
        else:
            # 재등장하는 단어의 인덱스
            index = word_to_index.get(word)
            # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더한다.
            bow[index] = bow[index] + 1

    return word_to_index, bow

In [24]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."

In [25]:
vocab, bow = build_bag_of_words(doc1)

In [26]:
print('Vocabulary :', vocab)
print('Bag of Words vector :', bow)

Vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
Bag of Words vector : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


- `bow` 출력에서 2가 나온 부분들을 주목해 보자.

### 다른 BoW 예

In [27]:
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'

In [28]:
vocab, bow = build_bag_of_words(doc2)
print('Vocabulary :', vocab)
print('Bag of Words vector :', bow)

Vocabulary : {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
Bag of Words vector : [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


In [29]:
doc3 = doc1 + ' ' + doc2
print(doc3)

정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다. 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.


In [30]:
vocab, bow = build_bag_of_words(doc3)
print('Vocabulary :', vocab)
print('Bag of Words vector :', bow)

Vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
Bag of Words vector : [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


#### doc3를 기반으로 하는 어휘를 기준으로 doc1, doc2의 BoW를 구한다면, 출력은 다음과 같을 것이다.
> 문서3 단어 집합에 대한 문서1 BoW : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]  
> 문서3 단어 집합에 대한 문서2 BoW : [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1] 

### <span style="color: crimson">[코딩🐞]</span> 위와 같은 출력이 나오도록 `build_bag_of_words` 를 수정해보자.

### CountVectorizer 클래스로 BoW 만들기

In [43]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print('bag of words vector :', vector.fit_transform(corpus).toarray()) 

# 각 단어의 인덱스가 어떻게 부여되었는지를 출력
print('vocabulary :',vector.vocabulary_)

bag of words vector : [[1 1 2 1 2 1]]
vocabulary : {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


#### vocabulary 보기가 영~ 불편해!
#### 어휘 사전을 오름차순으로 정리해서 출력/저장

In [48]:
# CountVectorizer 객체에서 어휘 딕셔너리를 value로 오름차순 정렬하여 반환
def sorted_vocab(vect):
    return dict(sorted(vect.vocabulary_.items(), key=lambda x: x[1], reverse=False))

In [52]:
word2idx = sorted_vocab(vector)
print(word2idx)

{'because': 0, 'know': 1, 'love': 2, 'want': 3, 'you': 4, 'your': 5}


- 단어 `I` 가 없어진 이유: CountVectorizer가 기본적으로 길이가 2이상인 문자에 대해서만 토큰으로 인식하기 때문
- CountVectorizer가 띄어쓰기만을 기준으로 토큰화하고 BoW를 만든다. 따라서 영어에서는 큰 문제가 없을 수 있으나, 한국어에서는 품질이 많이 떨어진다.

In [53]:
# doc3로 해보자.
print('bag of words vector :', vector.fit_transform([doc3]).toarray()) 
print('vocabulary :',sorted_vocab(vector))

bag of words vector : [[1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
vocabulary : {'기준으로': 0, '느끼는': 1, '느낀다': 2, '다르다': 3, '물가상승률과': 4, '물가상승률은': 5, '물가상승률을': 6, '발표하는': 7, '상품을': 8, '소비자가': 9, '소비자는': 10, '소비하는': 11, '정부가': 12, '주로': 13}


- '물가상승률과'과 '물가상승률을'이 다른 단어로 처리됨

### 불용어를 제거한 BoW

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

#### (1) 사용자가 직접 정의한 불용어 사용

In [55]:
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])
print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',sorted_vocab(vect))

bag of words vector : [[1 1 1 1 1]]
vocabulary : {'everything': 0, 'family': 1, 'important': 2, 'it': 3, 'thing': 4}


#### (2) CountVectorizer에서 제공하는 자체 불용어 사용

In [56]:
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words="english")
print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',sorted_vocab(vect))

bag of words vector : [[1 1 1]]
vocabulary : {'family': 0, 'important': 1, 'thing': 2}


#### (3) NLTK에서 지원하는 불용어 사용

In [57]:
text = ["Family is not an important thing. It's everything."]
stop_words = stopwords.words("english")
vect = CountVectorizer(stop_words=stop_words)
print('bag of words vector :',vect.fit_transform(text).toarray()) 
print('vocabulary :',sorted_vocab(vect))

bag of words vector : [[1 1 1 1]]
vocabulary : {'everything': 0, 'family': 1, 'important': 2, 'thing': 3}


# 3.2 TF-IDF

### 파이썬으로 TF-IDF 직접 구현하기

In [58]:
from math import log
import pandas as pd
 
docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 
 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
print('단어장의 크기 :', len(vocab))
print(vocab)

단어장의 크기 : 9
['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']


In [59]:
N = len(docs) # 문서의 총수
 
def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [60]:
result = []

# 각 문서에 대해서 아래 연산을 반복
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(t, d))
        
tf_ = pd.DataFrame(result, columns = vocab)  # 함수명 tf 와 구분하기 위해 underscore 사용
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [61]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [62]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


### 사이킷런을 이용한 DTM과 TF-IDF 실습

In [64]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())

# 각 단어와 맵핑된 인덱스 출력
print('vocabulary :',sorted_vocab(vector))

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
vocabulary : {'do': 0, 'know': 1, 'like': 2, 'love': 3, 'should': 4, 'want': 5, 'what': 6, 'you': 7, 'your': 8}


In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
# print(tfidfv.vocabulary_)
print(sorted_vocab(tfidfv))

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'do': 0, 'know': 1, 'like': 2, 'love': 3, 'should': 4, 'want': 5, 'what': 6, 'you': 7, 'your': 8}
